In [ ]:
!pip install -U pycld2

In [48]:
import re
import glob
import pandas as pd
import sweetviz
import random
import pandas as pd
import warnings
import unidecode
import numpy as np
import csv
import emoji
from typing import List
from flair.data import Token
import sys,os
import tqdm
import pycld2 as cld2

sys.path.append('../')

In [29]:
from preprocessing.amharicSegmenter import AmharicSegmenter

In [38]:
def find_url(string): 
    text = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',string)
    return "".join(text) # converting return value from list to string

In [39]:
def find_eng(string): 
    text = re.findall('[a-zA-Z0-9]',string)
    return "".join(text) # converting return value from list to string

In [40]:
def remove_url(data):
    return data.replace(find_url(data),"")

In [51]:
def remove_eng(data):
    return ' '.join(re.sub(u"[^ሀ-፼]", " ", data).split()).strip()
     

In [49]:
def remove_English_letters(content):
    for idx,cnt in enumerate(content):
        content[idx] = re.sub(r'\s*[A-Za-z]+\b','', cnt)
    return content

In [42]:
def remove_emoji(news):
    news = emoji.demojize(news)
    re.sub('(:\S+)(@\w+)','', news)
    return news

In [43]:
def duplicated_values_data(data):
    dup=[]
    columns=data.columns
    for i in data.columns:
        dup.append(sum(data[i].duplicated()))
    return pd.concat([pd.Series(columns),pd.Series(dup)],axis=1,keys=['Columns','Duplicate count'])

Before Removing Duplicated Sentences
   Columns  Duplicate count
0  Content              280
After Removing Duplicated Sentences
   Columns  Duplicate count
0  Content                0


In [44]:
sent_punct = []
word_punct = []
amseg = AmharicSegmenter(sent_punct,word_punct)

# Read each CSV files and write it to text file

In [53]:
remove = ['\xa087','\xa0', '|', '/','…','«','"','“','”','»','-',
          '$','%','^','&','*','›','’','‘','‹','<','>','`','´','~','=','+']
# remove output file if it exists, then append each csv to a single file output
outfile = "all_sentences.txt"
os.remove(outfile) if os.path.exists(outfile) else None
# all csv files, to controll progress par
allnews = []
for news in glob.glob("data/*.csv"):
    allnews.append(news)

# read each files and write to a file system   
for news in tqdm.tqdm(allnews,  position=0, leave=True):
    data = pd.read_csv(news, names=["ID","URL","Date","Media","Content"] ,encoding="utf-8")
    data = data[data.Content.duplicated()==False].reset_index()
    #data.Content = data.Content.apply(lambda x: remove_url(x))
    #data.Content = data.Content.apply(lambda x: remove_emoji(x))
    data.Content = data.Content.apply(lambda x: remove_eng(x))
    for token in remove:
        data.Content = data.Content.apply(lambda x: x.replace(token,' '))  
    text  = data.Content.to_string()
    with open (outfile,"a", encoding="utf-8") as all_sentences:
        for s in amseg.tokenize_sentence(text):
            isReliable, textBytesFound, details  = cld2.detect(s)
            if details[0][1] =='am':
                all_sentences.write(s.replace('\n', '')+'\n')



100%|██████████| 4/4 [00:06<00:00,  1.54s/it]


In [46]:
print(text)

0      መጭው አዲስ ዓመት 2021 እና አፍሪቃ\nየስደተኞች ቀውስ\nበትግራይ የተ...
1      የመቀለ ከተማ አሁን ስላለችበት ሁኔታ አዲሱ ከንቲባ ይናገራሉ\nበጥቅምት ...
2       \nየ አማርኛ ትግርኛ ሓተታ ትንተና ገጽ\nOpinions and Views...
3      በሳምንቱ ውስጥ የተከናወነ ወይም የሚከናወን አንድ ዓቢይ ፖለቲካዊ ጉዳይን...
4      DW English Live\nEuromaxx   Lifestyle Europe\n...
5                             የሥዕል ክምችት ድምፅ ቪድዮ\nዝግጅቶች\n
6      አዲስ ዓመት መጣ የቋጠርካትን አውጣ\nአዲስ ዓመት አዲስ በመሆኗ ትፈራለች...
7      በጣሊያን ኤምባሲ በቁም እስር ላይ የሚገኙ የቀድሞ የደርግ ባለስልጣናት እ...
8      5\n(0 votes)\n እንዳትነግረው ብዬ ብነግረው፤ እንዳትነግረው ብሎ ...
9      ቀረብ ኣዋጅ ተሃድሶ ክፍሊ 508\nshare\nመሕተሚ\nወኪል ዓለማዊ ሚድ...
10     ብቐጥታ\nብዛዕባ  ድምጺ ኣሜሪካ ቋንቋ ትግርኛ\nድምጺ ኣሜሪካ እቲ ዝዓበ...
11     5\n(1 Vote)\nለአርባ አምስት ዓመታት የኢትዮጵያና የትግራይ ህዝብ ...
12     የአፍቃሪው ደብዳቤ\n  አንቺ ማለት የልቤ እረፍት፣ የመንፈሴ ወሰን፣ የሰ...
13     በበረሃአንበጣሰብላቸውየወደመባቸው\n አርሶአደሮችምንይላሉ ?\n#በዚህ ዓመ...
14     አሉላ\n350ሺ ብር\nከትልቅ ጨዋታ በፊት በየትኛውም ቡድን ውስጥ ተጨዋቾ...
15      ትግራይን እንደ ዋዛ ከኢትዮጵያ መለየት አይቻልም \n   (የዛሬ 3 ሳም...
16     ብልጽግና ፓርቲ በወቅታዊ ጉዳዮች ላይ የሰጠው መግለጫ\nከሀገራዊ ለውጡ በ...
17        አገር ላይ አደጋ የተደቀነው አንቀ